In [2]:
import pandas as pd
import json
import sys

In [26]:
# nice JSON viewer, from https://stackoverflow.com/questions/18873066/pretty-json-formatting-in-ipython-notebook
import uuid
from IPython.display import display_javascript, display_html, display
import json

class RenderJSON(object):
    def __init__(self, json_data):
        if isinstance(json_data, dict):
            self.json_str = json.dumps(json_data)
        else:
            self.json_str = json_data
        self.uuid = str(uuid.uuid4())

    def _ipython_display_(self):
        display_html('<div id="{}" style="height: 600px; width:100%;"></div>'.format(self.uuid), raw=True)
        display_javascript("""
        require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {
        document.getElementById('%s').appendChild(renderjson(%s))
        });
        """ % (self.uuid, self.json_str), raw=True)

In [16]:
sys.path.append('./')
from _dstc_scripts.dataset_walker import dataset_walker
from _dstc_scripts import misc

In [49]:
dataset = dataset_walker("dstc2_dev", dataroot="_Data/dstc2_data", labels=True)

In [51]:
for call in dataset:
    if call.log['session-id'] == 'voip-be5694f464-20130328_125233':
        for n, (turn, label) in enumerate(call):
            print '-' * 20, n, '-' * 20
            #display(RenderJSON(turn)) #['output']))
            #display(RenderJSON(label))
            print json.dumps(turn['output'], indent=4)
            print json.dumps(label, indent=4)

-------------------- 0 --------------------
{
    "start-time": 0.0013, 
    "end-time": 9.04, 
    "transcript": "Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?", 
    "dialog-acts": [
        {
            "slots": [], 
            "act": "welcomemsg"
        }
    ], 
    "aborted": false
}
{
    "turn-index": 0, 
    "goal-labels": {
        "pricerange": "cheap"
    }, 
    "transcription": "i want to find a cheap re", 
    "method-label": "byconstraints", 
    "audio-file": "pt344x_0000904_0001369.wav", 
    "requested-slots": [], 
    "semantics": {
        "json": [
            {
                "slots": [
                    [
                        "pricerange", 
                        "cheap"
                    ]
                ], 
                "act": "inform"
            }
        ], 
        "cam": "inform(pricerange=cheap)"
    }
}
-------------------- 1 --------------------
{

**TODO**

* convert to dataframe? Using cam format of semantics
* look at MULTIWOZ format

In [37]:
with open('_dstc_scripts/config/ontology_dstc2.json', 'r') as f:
    ontology = json.load(f)

In [42]:
RenderJSON(json.dumps(ontology))

In [84]:
all_rows = []
for m, call in enumerate(dataset):
    this_dial_rows = []
    for n, (turn, label) in enumerate(call):
        sys_utt = turn['output']['transcript']
        sys_das_raw = turn['output']['dialog-acts']
        sys_das = []
        for this_da_raw in sys_das_raw:
            this_slots = '' if len(this_da_raw['slots']) == 0 else '{}={}'.format(this_da_raw['slots'][0][0],
                                                                                  this_da_raw['slots'][0][1])
            sys_das.append('{}({})'.format(this_da_raw['act'], this_slots))
        sys_das = '| '.join(sys_das)

        user_utt = label['transcription']
        user_das = label['semantics']['cam']
        user_das = user_das.replace(',', ', ')
        this_dial_rows.append((m, n, sys_utt, sys_das, user_utt, user_das, call.log['session-id']))
    all_rows.extend(this_dial_rows)

In [86]:
dial_df = pd.DataFrame(all_rows, columns='did tid sys_utt sys_sem usr_utt usr_sem call_id'.split())
#dial_df['did tid sys_utt usr_utt usr_sem sys_sem call_id'.split()]
dial_df

,did,tid,sys_utt,sys_sem,usr_utt,usr_sem,call_id
0,0,0,"Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?",welcomemsg(),i would like to find an expensive restaurant in the south part,"inform(pricerange=expensive, type=restaurant, area=south)",voip-f246dfe0f2-20130328_161556
1,0,1,What kind of food would you like?,request(slot=food),does not matter,inform(=dontcare),voip-f246dfe0f2-20130328_161556
2,0,2,There are restaurants in the expensive price range and the south of town . What type of food would you like?,request(slot=food)| impl-conf(pricerange=expensive)| impl-conf(area=south),any type of food is okay,inform(food=dontcare),voip-f246dfe0f2-20130328_161556
3,0,3,taj tandoori is a nice restaurant in the south of town serving indian food,offer(name=taj tandoori)| inform(food=indian)| inform(pricerange=expensive)| inform(area=south),what is the address,request(addr),voip-f246dfe0f2-20130328_161556
4,0,4,"Sure , taj tandoori is on 64 Cherry Hinton Road Cherry Hinton",offer(name=taj tandoori)| inform(addr=64 cherry hinton road cherry hinton),what is the phone number,request(phone),voip-f246dfe0f2-20130328_161556
5,0,5,The phone number of taj tandoori is 01223 412299 .,offer(name=taj tandoori)| inform(phone=01223 412299),what type of food,request(food),voip-f246dfe0f2-20130328_161556
6,0,6,taj tandoori serves indian food,offer(name=taj tandoori)| inform(food=indian),okay thank,thankyou(),voip-f246dfe0f2-20130328_161556
7,0,7,taj tandoori is a nice restaurant in the south of town in the expensive price range,offer(name=taj tandoori)| inform(pricerange=expensive)| inform(area=south),thank you good bye,thankyou()|bye(),voip-f246dfe0f2-20130328_161556
8,1,0,"Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?",welcomemsg(),i want to find a cheap re,inform(pricerange=cheap),voip-be5694f464-20130328_125233
9,1,1,the missing sock is a nice place in the east of town and the prices are cheap,offer(name=the missing sock)| inform(pricerange=cheap)| inform(area=east),i want to find the address,request(addr),voip-be5694f464-20130328_125233


In [80]:
pd.set_option('display.max_colwidth', -1)

In [102]:
def style_dial(df):
    display(df.style.applymap(lambda _:"color: red", subset=['sys_utt'])\
                    .applymap(lambda _:"color: blue", subset=['usr_utt']))

In [103]:
style_dial(dial_df.head())

,did,tid,sys_utt,sys_sem,usr_utt,usr_sem,call_id
0,0,0,"Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?",welcomemsg(),i would like to find an expensive restaurant in the south part,"inform(pricerange=expensive, type=restaurant, area=south)",voip-f246dfe0f2-20130328_161556
1,0,1,What kind of food would you like?,request(slot=food),does not matter,inform(=dontcare),voip-f246dfe0f2-20130328_161556
2,0,2,There are restaurants in the expensive price range and the south of town . What type of food would you like?,request(slot=food)| impl-conf(pricerange=expensive)| impl-conf(area=south),any type of food is okay,inform(food=dontcare),voip-f246dfe0f2-20130328_161556
3,0,3,taj tandoori is a nice restaurant in the south of town serving indian food,offer(name=taj tandoori)| inform(food=indian)| inform(pricerange=expensive)| inform(area=south),what is the address,request(addr),voip-f246dfe0f2-20130328_161556
4,0,4,"Sure , taj tandoori is on 64 Cherry Hinton Road Cherry Hinton",offer(name=taj tandoori)| inform(addr=64 cherry hinton road cherry hinton),what is the phone number,request(phone),voip-f246dfe0f2-20130328_161556


In [106]:
def get_dial(df, did):
    return df.query('did == @did')

In [108]:
style_dial(get_dial(dial_df, 24))

,did,tid,sys_utt,sys_sem,usr_utt,usr_sem,call_id
181,24,0,"Hello , welcome to the Cambridge restaurant system? You can ask for restaurants by area , price range or food type . How may I help you?",welcomemsg(),im looking for a cheap restaurant serving modern european type of food,"inform(pricerange=cheap, type=restaurant, food=""modern european"", task=find)",voip-597cfafdee-20130402_011135
182,24,1,golden house is in the cheap price range,offer(name=golden house)| inform(pricerange=cheap),serving modern european type of food,"inform(food=""modern european"")",voip-597cfafdee-20130402_011135
183,24,2,the lucky star is in the cheap price range,offer(name=the lucky star)| inform(pricerange=cheap),can i have the address and post code,"request(addr, postcode)",voip-597cfafdee-20130402_011135
184,24,3,"The post code of the lucky star is C.B 1, 7 D.Y",offer(name=the lucky star)| inform(postcode=c.b 1),can i have the address,request(addr),voip-597cfafdee-20130402_011135
185,24,4,the lucky star is a great restaurant,offer(name=the lucky star),can i have the address of the venue,request(addr),voip-597cfafdee-20130402_011135
186,24,5,"Sure , the lucky star is on Cambridge Leisure Park Clifton Way Cherry Hinton",offer(name=the lucky star)| inform(addr=cambridge leisure park clifton way cherry hinton),thank you good bye,thankyou()|bye(),voip-597cfafdee-20130402_011135


**TODO**

- write starter framework for NLU with regular expressions?
- or just give hint to go from ontology to reg exps?
- look at scoring function! How do you evaluate correctness of NLU? How did the DSTC people do this?
- they are only allowed to look at the train part of the corpus. evaluate during development on devel.
- we will evaluate in class on test.